In [265]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [266]:
results = pd.read_csv('optimal_bidding_bis/results/mpc_results.csv')
results

,Unnamed: 0,battery_bid_price,battery_bid_power_gen,battery_bid_power_load,battery_bid_cleared_power_gen,battery_bid_cleared_power_load,clearing_price,soe,timestamp
0,0,54.90,0.0,300.0,0,0,40.0,0,2018-06-01 04:30:00
1,1,56.19,0.0,300.0,0,0,40.0,0,2018-06-01 05:00:00
2,2,58.73,0.0,300.0,0,0,40.0,0,2018-06-01 05:30:00
3,3,62.98,0.0,300.0,0,0,56.0,0,2018-06-01 06:00:00
4,4,74.24,0.0,300.0,0,0,56.0,0,2018-06-01 06:30:00
...,...,...,...,...,...,...,...,...,...
7283,7283,95.60,0.0,300.0,0,0,67.0,0,2018-10-30 22:00:00
7284,7284,152.80,0.0,0.0,0,0,67.0,0,2018-10-30 22:30:00
7285,7285,134.27,0.0,0.0,0,0,67.0,0,2018-10-30 23:00:00
7286,7286,111.20,0.0,300.0,0,0,67.0,0,2018-10-30 23:30:00


In [267]:
results['timestamp'] = pd.to_datetime(results['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
results['battery_bid_price'] = pd.to_numeric(results['battery_bid_price'], errors='coerce')
results['battery_bid_power_gen'] = pd.to_numeric(results['battery_bid_power_gen'], errors='coerce')
results['battery_bid_power_load'] = pd.to_numeric(results['battery_bid_power_load'], errors='coerce')
results['battery_bid_cleared_power_gen'] = pd.to_numeric(results['battery_bid_cleared_power_gen'], errors='coerce')
results['battery_bid_cleared_power_load'] = pd.to_numeric(results['battery_bid_cleared_power_load'], errors='coerce')
results['clearing_price'] = pd.to_numeric(results['clearing_price'], errors='coerce')
results['soe'] = pd.to_numeric(results['soe'], errors='coerce')

start = pd.Timestamp(year=2018,
                     month=7,
                     day=14,
                     hour=0,
                     minute=0)
end = pd.Timestamp(year=2018,
                     month=7,
                     day=8,
                     hour=0,
                     minute=0)

# results = results[(results['timestamp'] > start) & (results['timestamp'] < end)]
#results = results[(results['timestamp'] < end)]

results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7288 entries, 0 to 7287
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Unnamed: 0                      7288 non-null   int64         
 1   battery_bid_price               7288 non-null   float64       
 2   battery_bid_power_gen           7288 non-null   float64       
 3   battery_bid_power_load          7288 non-null   float64       
 4   battery_bid_cleared_power_gen   7288 non-null   int64         
 5   battery_bid_cleared_power_load  7288 non-null   int64         
 6   clearing_price                  7288 non-null   float64       
 7   soe                             7288 non-null   int64         
 8   timestamp                       7288 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4)
memory usage: 512.6 KB


In [273]:
def compute_reward(results):
    reward = []
    for i in range(len(results)):
        r = - results['battery_bid_cleared_power_load'].iloc[i] * results['clearing_price'].iloc[i]
        if results['battery_bid_price'].iloc[i] <= results['clearing_price'].iloc[i]:
            r += - results['battery_bid_cleared_power_gen'].iloc[i] * results['clearing_price'].iloc[i]
        reward.append(r)
    results['reward'] = reward
    return results

In [274]:
results = compute_reward(results)
results

,Unnamed: 0,battery_bid_price,battery_bid_power_gen,battery_bid_power_load,battery_bid_cleared_power_gen,battery_bid_cleared_power_load,clearing_price,soe,timestamp,reward
0,0,54.90,0.0,300.0,0,0,40.0,0,2018-06-01 04:30:00,0.0
1,1,56.19,0.0,300.0,0,0,40.0,0,2018-06-01 05:00:00,0.0
2,2,58.73,0.0,300.0,0,0,40.0,0,2018-06-01 05:30:00,0.0
3,3,62.98,0.0,300.0,0,0,56.0,0,2018-06-01 06:00:00,0.0
4,4,74.24,0.0,300.0,0,0,56.0,0,2018-06-01 06:30:00,0.0
...,...,...,...,...,...,...,...,...,...,...
7283,7283,95.60,0.0,300.0,0,0,67.0,0,2018-10-30 22:00:00,0.0
7284,7284,152.80,0.0,0.0,0,0,67.0,0,2018-10-30 22:30:00,0.0
7285,7285,134.27,0.0,0.0,0,0,67.0,0,2018-10-30 23:00:00,0.0
7286,7286,111.20,0.0,300.0,0,0,67.0,0,2018-10-30 23:30:00,0.0


In [275]:
cumulative_reward = results['reward'].rolling(window=48).mean()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results['timestamp'], y=cumulative_reward, name='reward'), secondary_y=False)

# Add figure title
fig.update_layout(
    title_text="Cumulative Reward",
    # xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Cumulative Reward")
fig.show()
# fig.write_image('cum_reward_mpc.png')

In [271]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'], y=results['soe'], name='soe'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'], y=results['clearing_price'], name='clearing price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Electricity Demand (MW)", secondary_y=False)
fig.update_yaxes(title_text="Electricity Price ($/MW)", secondary_y=True)

fig.show()

In [272]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_cleared_power_load'], name='energy power load'), secondary_y=False)
fig.add_trace(go.Bar(x=results['timestamp'].index, y=results['battery_bid_cleared_power_gen'], name='energy power gen'), secondary_y=False)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['battery_bid_price'], name='battery bid price'), secondary_y=True)
fig.add_trace(go.Scatter(x=results['timestamp'].index, y=results['clearing_price'], name='clearing price'), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Mean of demand and price for a day",
    xaxis_rangeslider_visible=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Energy Power (MW)", secondary_y=False)
fig.update_yaxes(title_text="Energy Price ($/MW)", secondary_y=True)

fig.show()